## Modèle Xgboost

### Entrainement

In [ ]:
X_train1=X_train.copy()
del X_train1['building_id']

In [ ]:
X_test1=X_test.copy()
del X_test1['building_id']

In [ ]:
X_train1.to_csv (r'/content/drive/MyDrive/Kaggle/X_train1.csv', index = False, header=True)
X_test1.to_csv (r'/content/drive/MyDrive/Kaggle/X_test1.csv', index = False, header=True)
Y_train.to_csv (r'/content/drive/MyDrive/Kaggle/Y_train.csv', index = False, header=True)
Y_test.to_csv (r'/content/drive/MyDrive/Kaggle/Y_test.csv', index = False, header=True)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.stats
import gc
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
X_train1 = pd.read_csv('/content/drive/MyDrive/Kaggle/X_train1.csv')#chemin Olivier

In [ ]:
X_test1 = pd.read_csv('/content/drive/MyDrive/Kaggle/X_test1.csv') #cHemin Olivier

In [ ]:
Y_train = pd.read_csv('/content/drive/MyDrive/Kaggle/Y_train.csv')#chemin Olivier

In [ ]:
Y_test = pd.read_csv('/content/drive/MyDrive/Kaggle/Y_test.csv')#chemin Olivier

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
X_train1=reduce_mem_usage(X_train1)
X_test1=reduce_mem_usage(X_test1)

Memory usage after optimization is: 534.56 MB
Decreased by 85.9%
Memory usage after optimization is: 294.46 MB
Decreased by 85.9%


In [ ]:
Y_train=reduce_mem_usage(Y_train)
Y_test1=reduce_mem_usage(Y_test)

Memory usage after optimization is: 24.86 MB
Decreased by 75.0%
Memory usage after optimization is: 13.70 MB
Decreased by 75.0%


In [ ]:
! sudo pip install xgboost

In [ ]:
import xgboost as XGB

In [ ]:
params = {
    "learning_rate": 0.05,
    "objective": "reg:squarederror",
    "metric": "rmse",
    "reg_lambda": 2
    "tree_method": "approx"
}

In [ ]:
XGB = XGB(**params)

In [ ]:
train = XGB.DMatrix(data=X_train1,label=Y_train)
valid = XGB.DMatrix(data=X_test1,label=Y_test)

In [ ]:
eval_list = [(valid, 'eval')]

In [ ]:
XGB_Reg=XGB.train(train
        eval_set=eval_list,
        eval_metric='rmse',
        verbose=True)

In [ ]:
XGB_Reg.save_model('/content/drive/MyDrive/Kaggle/XGB_Reg.txt') 

In [ ]:
pickle.dump(XGB_Reg, open("pima.pickle.dat", "wb"))

### Prediction et soumission

In [ ]:
! mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
! cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! unzip '/content/test.csv.zip' -d test
! unzip '/content/weather_test.csv.zip' -d weather_test

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**3
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**3
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
test = reduce_mem_usage(test)

In [ ]:
weather_test = reduce_mem_usage(weather_test)

In [ ]:
building_metadata = reduce_mem_usage(building_metadata)

In [ ]:
test =test.merge(building_metadata, on='building_id', how='left')
alltest= test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
del test, weather_test,building_metadata
gc.collect()

In [ ]:
alltest = reduce_mem_usage(alltest)

In [ ]:
alltest.timestamp=pd.to_datetime(alltest['timestamp'])

In [ ]:
(alltest.isna().sum()/alltest.shape[0]).sort_values(ascending=False)

In [ ]:
alltest.to_csv (r'/content/drive/MyDrive/Kaggle/alltest.csv', index = False, header=True)

In [ ]:
data_test = preProcecing_df(alltest)

In [ ]:
data_test['meter'] = pd.Categorical(data_test['meter']).rename_categories({0: 'electricity', 
                                                                   1: 'chilledwater',
                                                                   2: 'steam', 
                                                                   3: 'hotwater'})

In [ ]:
reduce_mem_usage(data_test)

In [ ]:
data_test["building_id"] = data_test["building_id"].astype('category')
data_test["site_id"] = data_test["site_id"].astype('category')
data_test["primary_use"] = data_test["primary_use"].astype('category')
data_test["saison"] = data_test["saison"].astype('category')
data_test["heureDiscredite"] = data_test["heureDiscredite"].astype('category')
data_test["year_built"] = data_test["year_built"].astype('category')

In [ ]:
data_test.to_csv (r'/content/drive/MyDrive/Kaggle/data_test.csv', index = False, header=True)

In [ ]:
listColumns= [ 'wind_direction',  'dew_temperature',
              'air_temperature', 'floor_count', 'square_feet']

gc.collect()

In [ ]:
def encodeur(df): 
    X_Encod=pd.concat([df, pd.get_dummies(df["primary_use"], dtype=int) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["saison"], dtype=int) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["heureDiscredite"], dtype=int) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["meter"], dtype=int) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["year_built"], dtype=int) ], axis=1)
    reduce_mem_usage(X_Encod)

    for col in ["primary_use",'year_built', "saison", "heureDiscredite", 'meter']:
        del X_Encod[col]


    return X_Encod

In [ ]:
X_final = encodeur(data_test)

In [ ]:
X_final = minMax(X_final, listColumns, scaler)
del data_test

In [ ]:
X_final = X_final.drop(["row_id"],axis=1)

In [ ]:
del X_final['building_id']

In [ ]:
X_final["site_id"] = X_final["site_id"].astype('int')

In [ ]:
X_final.to_csv (r'/content/drive/MyDrive/Kaggle/X_final.csv', index = False, header=True)

In [ ]:
X_final  = pd.read_csv('/content/drive/MyDrive/Kaggle/X_final.csv')

In [ ]:
reduce_mem_usage(X_final)

In [ ]:
X_final.info()

In [ ]:
#cols = list(X_final.columns.values)

In [ ]:
#cols=[ 'site_id', 'square_feet', 'floor_count', 'air_temperature', 'dew_temperature', 'wind_direction', 'week_end', 'Education', 'Entertainment/public assembly', 'Food sales and service', 'Healthcare', 'Lodging/residential', 'Manufacturing/industrial', 'Other', 'Parking', 'Public services', 'Religious worship', 'Retail', 'Services', 'Technology/science', 'Utility', 'Warehouse/storage', 'automne', 'ete', 'hiver', 'nuit', 'tard', 'chilledwater','electricity', 'steam', 'yearB_q1', 'yearB_q2', 'yearB_q3']

In [ ]:
#X_final=X_final[cols]

In [ ]:
loaded_model = pickle.load(open("pima.pickle.dat", "rb"))

In [ ]:
X_final.info()

In [ ]:
X_final1=X_final.iloc[0:8000000,0:38]
reduce_mem_usage(X_final1)
X_final1 = XGB.DMatrix(data=X_final1)

In [ ]:
Y_pred_final1 = loaded_model.predict(X_final1,ntree_limit= loaded_model.best_ntree_limit)

In [ ]:
Y_pred_final1 = pd.DataFrame(data=Y_pred_final1)
Y_pred_final1=Y_pred_final1.to_csv('/content/Y_pred_final1.csv', index=False)

In [ ]:
del X_final1
del Y_pred_final1

In [ ]:
X_final2=X_final.iloc[8000000:16000000,0:38]
reduce_mem_usage(X_final2)
X_final2 = XGB.DMatrix(data=X_final2)

In [ ]:
loaded_model.best_ntree_limit

200

In [ ]:
Y_pred_final2 = loaded_model.predict(X_final2,ntree_limit=loaded_model.best_ntree_limit)

In [ ]:
Y_pred_final2 = pd.DataFrame(data=Y_pred_final2)
Y_pred_final2 = Y_pred_final2.to_csv('/content/Y_pred_final2.csv', index=False)

In [ ]:
del X_final2
del Y_pred_final2

In [ ]:
X_final3=X_final.iloc[16000000:24000000,0:38]
reduce_mem_usage(X_final3)
X_final3 = XGB.DMatrix(data=X_final3)

Memory usage after optimization is: 0.32 MB
Decreased by 0.0%


In [ ]:
Y_pred_final3 = loaded_model.predict(X_final3,ntree_limit= loaded_model.best_ntree_limit)

In [ ]:
Y_pred_final3 = pd.DataFrame(data=Y_pred_final3)
Y_pred_final3 = Y_pred_final3.to_csv('/content/Y_pred_final3.csv', index=False)

In [ ]:
del X_final3
del Y_pred_final3

In [ ]:
X_final4=X_final.iloc[24000000:32000000,0:38]
reduce_mem_usage(X_final4)
X_final4 = XGB.DMatrix(data=X_final4)

In [ ]:
Y_pred_final4 = loaded_model.predict(X_final4,ntree_limit= loaded_model.best_ntree_limit)

In [ ]:
Y_pred_final4 = pd.DataFrame(data=Y_pred_final4)
Y_pred_final4 = Y_pred_final4.to_csv('/content/Y_pred_final4.csv', index=False)

In [ ]:
del X_final4
del Y_pred_final4

In [ ]:
X_final5=X_final.iloc[32000000:41697601,0:38]
reduce_mem_usage(X_final5)
X_final5 = XGB.DMatrix(data=X_final5)

In [ ]:
Y_pred_final5 = loaded_model.predict(X_final5,ntree_limit= loaded_model.best_ntree_limit)

In [ ]:
Y_pred_final5 = pd.DataFrame(data=Y_pred_final5)
Y_pred_final5 = Y_pred_final5.to_csv('/content/Y_pred_final5.csv', index=False)

In [ ]:
del X_final5
del Y_pred_final5

In [ ]:
Y_pred_final1 = pd.read_csv('/content/Y_pred_final1.csv')

In [ ]:
Y_pred_final2 = pd.read_csv('/content/Y_pred_final2.csv')

In [ ]:
reduce_mem_usage(Y_pred_final1)

In [ ]:
reduce_mem_usage(Y_pred_final2)

In [ ]:
Y_pred_final3 = pd.read_csv('/content/Y_pred_final3.csv')

In [ ]:
reduce_mem_usage(Y_pred_final3)

In [ ]:
Y_pred_final4 = pd.read_csv('/content/Y_pred_final4.csv')

In [ ]:
reduce_mem_usage(Y_pred_final4)

In [ ]:
Y_pred_final5 = pd.read_csv('/content/Y_pred_final5.csv')

In [ ]:
reduce_mem_usage(Y_pred_final5)

In [ ]:
del X_final

In [ ]:
Y_pred_finalXGB_df = pd.concat([Y_pred_final1, Y_pred_final2,Y_pred_final3,Y_pred_final4,Y_pred_final5])

In [ ]:
Y_pred_finalXGB_df.info()

In [ ]:
Y_pred_finalXGB_df.to_csv (r'/content/drive/MyDrive/Kaggle/Y_pred_finalXGB_df.csv', index = False, header=True)

In [ ]:
del Y_pred_finalXGB_df

In [ ]:
Y_pred_finalXGB=pd.read_csv('/content/drive/MyDrive/Kaggle/Y_pred_finalXGB_df.csv')

In [ ]:
reduce_mem_usage(Y_pred_finalXGB)

In [ ]:
Y_pred_finalXGB=np.exp(Y_pred_finalXGB)

In [ ]:
Y_pred_finalXGB.loc[ (X_final.site_id==0)&(X_final.electricity==1) ] /=0.2931

In [ ]:
submission  = pd.read_csv('/content/drive/MyDrive/Kaggle/sample_submission.csv')
submission['meter_reading'] = Y_pred_finalXGB
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('/content/drive/MyDrive/Kaggle/submission.csv', index=False)

In [ ]:
! kaggle competitions submit -c ashrae-energy-prediction -f '/content/drive/MyDrive/Kaggle/submission.csv' -m "Third submission using Xgboost algorithm"

100% 579M/579M [00:22<00:00, 26.7MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III